## Rally

### Verificando instalação

In [ ]:
!python -V
# Python 3.6.5

import requests
import pyral
pyral.__version__
# (1, 4, 1)

### Obtem token

> Código gerado pelo [Postman](https://www.getpostman.com/)

In [ ]:
import requests

url = "https://rally1.rallydev.com/slm/webservice/v2.0/security/authorize"

headers = {
    'Authorization': "Basic <user-password-digest>,
    'Cache-Control': "no-cache",
    'Postman-Token': "6c82ea0f-f73b-47f2-8b68-dd64923078aa"
    }

response = requests.request("GET", url, headers=headers)

print(response.text)

In [ ]:
import json

json_data = json.loads(response.text)
APIKEY = (json_data['OperationResult']['SecurityToken'])
print (APIKEY)

### Connecta no servidor

In [ ]:
import sys

from pyral import Rally, rallyWorkset

SERVER='rally1.rallydev.com'
WORKSPACE='your-workspace'
PROJECT='your-project'

rally = Rally(SERVER, apikey=APIKEY, workspace=WORKSPACE, project=PROJECT)

print ('Connected...')

### Retorna todos os workspaces

In [ ]:
workspaces = rally.getWorkspaces()
for wksp in workspaces:
    print ("%s %s" % (wksp.oid, wksp.Name))
    projects = rally.getProjects(workspace=wksp.Name)
    for proj in projects:
        print ("    %12.12s  %s" % (proj.oid, proj.Name))

### Recupera a lista de usuários de um workspace

In [ ]:
all_users = rally.getAllUsers()
print ("ObjectID;Name;UserName;Disabled")
for user in all_users:
    tz = user.UserProfile.TimeZone or 'default'
    role = user.Role or '-No Role-'
    values = (int(user.oid), user.Name, user.UserName, user.Disabled)
    print ("%s;%s;%s;%s" % values)

### Buscar usuário por username

In [ ]:
USERNAME = '<user-name>'
query_criteria = 'UserName = "%s"' % USERNAME
response = rally.get('User', fetch=True, query=query_criteria)
if response.errors:
    sys.stdout.write("\n".join(errors))
    sys.exit(1)
for user in response:  # there should only be one qualifying TestCase
    print ("%s %s %s" % (user.Name, user.Disabled, user.oid))

### Atualizar usuário por id

In [ ]:
# target_date must be in ISO-8601 format "YYYY-MM-DDThh:mm:ssZ"

user_data = { 
              "ObjectID" : "<object-id>",
              "Disabled" : "true"
            }
try:
    user = rally.update('User', user_data)
except Exception as details:
    sys.stderr.write('ERROR: %s \n' % details)
    sys.exit(1)

print ("User %s updated to Disabled=%s" % (user.UserName, user.Disabled))

## Referências

* [RallyRestToolkitForPython](https://github.com/RallyTools/RallyRestToolkitForPython#getting-started)
* [Rally Web Services API Documentation v2.0](https://rally1.rallydev.com/slm/doc/webservice/)